# Imports

In [1]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 
time: 574 µs (started: 2023-01-03 14:29:44 +00:00)


In [2]:
import numpy as np
import math
import statistics
import random
import secrets
from numpy.random import default_rng

time: 7.3 ms (started: 2023-01-03 14:29:44 +00:00)


# Global Variables

In [3]:
POPULATION = []
NEW_POPULATION = []
length_population = 100
length_chromosome = 12
rate_crossover = 90
rate_mutation = 50
MinLT = 1
MaxLT = 3

time: 1.39 ms (started: 2023-01-03 14:29:44 +00:00)


# Class

In [4]:
class Chromosome:
  score = 0
  idade = 0
  tempo_vida = 0
  
  def __init__(self,schema):
    self.schema = schema
  
  def __str__(self):
    toString = ''
    for ind in self.schema:
      toString += ind  
    return toString

time: 1.65 ms (started: 2023-01-03 14:29:44 +00:00)


# Selection

In [5]:
def avaregePopulation(population):
  sum = 0
  for i in population:
    sum += i.score
  
  if sum == 0:
    return 0
  else:
    return sum/ len(population)

time: 6.42 ms (started: 2023-01-03 14:29:44 +00:00)


In [6]:
def tempoVidaMedio(population):
  sum = 0
  for i in population:
    sum += i.tempo_vida
  
  return sum/ len(population)

time: 1.08 ms (started: 2023-01-03 14:29:44 +00:00)


In [7]:
def minFuc(n , score, mediaPopulation):
  p = MinLT + round(n * (score/mediaPopulation))
  return p

time: 956 µs (started: 2023-01-03 14:29:44 +00:00)


In [8]:
def vida(popu):
  n = tempoVidaMedio(POPULATION)
  mediaPopulation = avaregePopulation(POPULATION)

  for cromo in popu:
    # print("Entrei aqui")
    res = minFuc(n,cromo.score, mediaPopulation)
    # print(type(res))
    cromo.tempo_vida = min(res, MaxLT)


time: 1.71 ms (started: 2023-01-03 14:29:44 +00:00)


In [9]:
def selection(population, new_population):

  result = []
  merged_list = []
  merged_list.extend(population)
  merged_list.extend(new_population)

  for ind in merged_list:
    if ind.idade < ind.tempo_vida:
      result.append(ind)

  result.sort(key=lambda schema: schema.score, reverse = True)

  return result

time: 1.92 ms (started: 2023-01-03 14:29:44 +00:00)


# Mutation

In [10]:
def mutation(population):
  array = []

  for ind in population:
    array_2 = []
    # print(f"Entrei com esse schema{ind.schema} score é {ind.score}")
    for i,ind_schema in enumerate(ind.schema):
      yes = np.random.randint(0,100)
      
      if yes <= rate_mutation:
        if ind_schema == '0':
          array_2.append('1')
        elif ind_schema == '1':
          array_2.append('0')
      else:
        array_2.append(ind_schema)
  
    array.append(Chromosome(array_2))

  return array

time: 7.41 ms (started: 2023-01-03 14:29:45 +00:00)


# CrossOver

In [11]:
def topParents(population):
  population.sort(key=lambda schema: schema.score, reverse = True)
  re = []

  for ind in population:
    re.append(ind.schema)

  return re[:25]

time: 8 ms (started: 2023-01-03 14:29:45 +00:00)


In [12]:
def crossOver(population):
  parents = topParents(population)
  # print(len(parents))

  while len(NEW_POPULATION) < length_population:
    father = parents[np.random.randint(0,len(parents))]
    mother = parents[np.random.randint(0,len(parents))]   
    probability_crossover = np.random.randint(0,100)

    if probability_crossover <= rate_crossover:
      if father != mother:
        child = []
        cut = np.random.randint(1,length_chromosome)
        child.append(father[:cut] + mother[cut:])
        child.append(mother[:cut] + father[cut:])
        
        for downward in child: 
          NEW_POPULATION.append(Chromosome(downward))

time: 3.35 ms (started: 2023-01-03 14:29:45 +00:00)


# Score

In [13]:
def score(population_test):
  for ind in population_test:
    ind.score= ind.schema.count('1')

time: 953 µs (started: 2023-01-03 14:29:45 +00:00)


# Init Population

In [14]:
def random(): 
  array2 = []

  rng = default_rng()
  numbers = rng.choice(range(0, 2), size= 2, replace=False)
  
  array2.append(str(numbers[0]))
  array2.append(str(numbers[1]))
  
  return array2

time: 1.48 ms (started: 2023-01-03 14:29:45 +00:00)


In [15]:
def init_population():
  for _ in range(length_population):
    array = []
    for _ in range(int(length_chromosome/2)):
      array.extend(random())
    POPULATION.append(Chromosome(array))


time: 1.42 ms (started: 2023-01-03 14:29:45 +00:00)


# Diversidade da Populacao

In [16]:
def population_diversity(population):
  array_2 = []
  for i in population:
    array_2.append(str(i))
  
  return (len(set(array_2))/len(population))*100

time: 4.85 ms (started: 2023-01-03 14:29:45 +00:00)


# Main

In [17]:
count = 0
array_geracao = []
list_more_adaptado = []
array_diversidade = []

while count < 1:
  flag = False
  generation = 0
  POPULATION.clear()
  NEW_POPULATION.clear()
  init_population()
  score(POPULATION)
  vida(POPULATION)

  while True:
    score(POPULATION)
    array_diversidade.append(population_diversity(POPULATION))
    crossOver(POPULATION)
    score(NEW_POPULATION)
    NEW_POPULATION = mutation(NEW_POPULATION)
    score(NEW_POPULATION)
    vida(NEW_POPULATION)
    POPULATION = selection(POPULATION,NEW_POPULATION)
    NEW_POPULATION.clear()
    
    if POPULATION[0].score == 12:
      flag = True
    
    for ind in POPULATION:
      ind.idade += 1

    # list_more_adaptado.append(POPULATION[0].score)

    if flag:
      print("===================================================================")
      print(f'Individuo: {POPULATION[0].schema} e o score dele {POPULATION[0].score} geracao {generation}')
      print("===================================================================")
      array_geracao.append(generation)
      break
      
    generation += 1
  count += 1
  
print(f'media das 100 geracao {statistics.mean(array_geracao)}')

Individuo: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'] e o score dele 12 geracao 109
media das 100 geracao 109
time: 1.15 s (started: 2023-01-03 14:29:45 +00:00)


In [18]:
print(array_geracao)

[109]
time: 594 µs (started: 2023-01-03 14:29:46 +00:00)


In [19]:
# print(list_more_adaptado)

time: 433 µs (started: 2023-01-03 14:29:46 +00:00)


In [20]:
print(array_diversidade)

[46.0, 72.0, 98.44559585492227, 99.51219512195122, 98.69565217391305, 97.2027972027972, 96.21993127147766, 96.57534246575342, 97.23183391003461, 97.58620689655172, 97.9020979020979, 98.26388888888889, 97.59450171821305, 98.63013698630137, 96.93877551020408, 96.18055555555556, 94.23728813559322, 93.51535836177474, 96.27118644067797, 96.23287671232876, 97.61904761904762, 97.26027397260275, 96.91780821917808, 96.5986394557823, 95.50173010380622, 95.47038327526133, 95.6081081081081, 95.53264604810997, 95.8904109589041, 95.54794520547945, 97.24137931034483, 97.9381443298969, 97.95221843003414, 96.9283276450512, 96.5986394557823, 98.9795918367347, 98.29931972789116, 96.94915254237289, 94.86301369863014, 94.15807560137456, 96.21993127147766, 95.90443686006826, 95.22184300341297, 95.91836734693877, 94.77351916376307, 96.57534246575342, 95.578231292517, 95.51724137931035, 93.89830508474576, 94.21768707482994, 96.98996655518395, 97.2972972972973, 96.86411149825784, 95.6081081081081, 96.598639455